[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/quick-tour/namespacing.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/quick-tour/namespacing.ipynb)

# Namespacing with Pinecone

Namespacing is a neat feature in a Pinecone service that allows you to partition your data in an index. When you read from or write to a namespace in an index, you only access data in that particular namespace. In other words, data from two namespaces may have the same ids but different values. Namespacing is useful when you want to reuse the same data processing pipeline but query only a subset of your data.
For example, when you are building a movie recommender system, you could use namespacing to separate recommendations by genre. Another way is to use metadata instead of namespaces for recommendations by genre.

## Prerequisites

A paid Pinecone account as namespaces are not available on the free starter tier.

Install dependencies.

Set up Pinecone.

In [ ]:
import os
import lantern_pinecone as pinecone

LANTERN_DB_URL = os.environ.get('LANTERN_DB_URL') or 'postgres://postgres@localhost:5432'
pinecone.init(LANTERN_DB_URL)

## Creating the Index

In [2]:
index_name = "pinecone-namespacing"

In [3]:
import time

# Delete index if exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

# Create an index
pinecone.create_index(name=index_name, dimension=2, metric="euclidean")

# wait for index to be ready before connecting
while not pinecone.describe_index(index_name).status['ready']:
    time.sleep(1)

pinecone.list_indexes()

['pinecone-namespacing']

In [4]:
# Connect to the index
index = pinecone.Index(index_name=index_name)

### Generate movie data

In [5]:
# Generate some data
import pandas as pd

df = pd.DataFrame()
df["id"] = ["Wall-E", "Up", "Ratatouille", "Toy Story"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.]]
df

,id,vector
0,Wall-E,"[1.0, 1.0]"
1,Up,"[2.0, 2.0]"
2,Ratatouille,"[3.0, 3.0]"
3,Toy Story,"[4.0, 4.0]"


### Insert vectors without specifying a namespace

In [6]:
# Insert vectors without specifying a namespace
index.upsert(vectors=zip(df.id, df.vector))
index.describe_index_stats()

{'dimensions': 2,
 'index_fullness': 1.0,
 'total_count': 4,
 'namespaces': {'': {'vector_count': 4}}}

### Insert vectors into a namespace

In [7]:
romantic_comedies = ["Wall-E", "Ratatouille"]
romcom_df = df[df.id.isin(romantic_comedies)]
romcom_df

,id,vector
0,Wall-E,"[1.0, 1.0]"
2,Ratatouille,"[3.0, 3.0]"


In [8]:
# Insert vectors into a namespace
index.upsert(vectors=zip(romcom_df.id, romcom_df.vector), namespace="romantic-comedy")
index.describe_index_stats()

{'dimensions': 2,
 'index_fullness': 1.0,
 'total_count': 6,
 'namespaces': {'': {'vector_count': 4},
  'romantic-comedy': {'vector_count': 2}}}

### Query top-3 results, without a namespace

In [9]:
query_results = index.query(df[df.id == "Wall-E"].vector[0], top_k=3)
query_results

{'namespace': '',
 'matches': [{'id': 'Wall-E', 'score': 0.0, 'values': None, 'metadata': None},
  {'id': 'Up', 'score': 2.0, 'values': None, 'metadata': None},
  {'id': 'Ratatouille', 'score': 8.0, 'values': None, 'metadata': None}]}

### Query top-3 results, with a namespace

We should expect to see only romantic comedies in the query results.

In [10]:
query_results = index.query(
    df[df.id == "Wall-E"].vector[0], top_k=3, namespace="romantic-comedy"
)
query_results

{'namespace': 'romantic-comedy',
 'matches': [{'id': 'Wall-E', 'score': 0.0, 'values': None, 'metadata': None},
  {'id': 'Ratatouille', 'score': 8.0, 'values': None, 'metadata': None}]}

### Delete the index

In [11]:
# delete the index
pinecone.delete_index(index_name)